In [2]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

In [3]:
# Define the path to your CSV file
file_path = "data/processed/processed_data.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path, dtype="str")

total_national_households = df["OBS_VALUE"].astype(float).max()

print(total_national_households)

1780527.0


In [4]:
household_composition_classification = pd.read_csv(
    "data/classifications/household_composition_classification.csv", dtype="str"
)

household_composition_classification_codes = household_composition_classification[
    ["hhld_comp_l1_code", "hhld_comp_l2_code", "hhld_comp_l3_code"]
]

household_composition_classification_codes_all_levels = (
    household_composition_classification_codes.melt()
    .drop_duplicates()
    .rename(columns={"variable": "hhld_comp_level", "value": "household_composition_code"})
)

household_composition_classification_codes_all_levels.loc[
    household_composition_classification_codes_all_levels["household_composition_code"]
    == "99999",
    "hhld_comp_level",
] = "Total"

household_composition_classification_codes_all_levels.loc[
    household_composition_classification_codes_all_levels["household_composition_code"]
    == "77777",
    "hhld_comp_level",
] = "Total stated"

household_composition_classification_codes_all_levels.dropna(inplace=True)

In [5]:
number_bedrooms_classification = pd.read_csv(
    "data/classifications/number_bedrooms_classification.csv", dtype='str'
)


number_bedrooms_all_levels = number_bedrooms_classification.rename(
    columns={"num_bedrooms_descriptor": "num_bedrooms_level", "num_bedrooms_code": "num_bedrooms"},
    inplace=False,
)

number_bedrooms_all_levels["num_bedrooms_level"] = 'num_bedrooms_l1_code'

number_bedrooms_all_levels.loc[
    number_bedrooms_all_levels["num_bedrooms"] == "777", "num_bedrooms_level"
] = "Total stated"
number_bedrooms_all_levels.loc[
    number_bedrooms_all_levels["num_bedrooms"] == "999", "num_bedrooms_level"
] = "Total"

In [6]:
sa2_ta_concordance = pd.read_csv("data/classifications/sa2_to_ta_map.csv", dtype='str').drop_duplicates()

areas_all_levels = (
    sa2_ta_concordance.melt()
    .drop_duplicates()
    .rename(columns={"variable": "area_level", "value": "area_code"})
)

areas_all_levels.loc[areas_all_levels["area_code"] == "999999", "area_level"] = "Total"

areas_all_levels.drop_duplicates(inplace=True)

In [7]:
unique_hhld_comps = df["household_composition_code"].drop_duplicates()


unique_hhld_comps = unique_hhld_comps[
    ~unique_hhld_comps.isin(
        household_composition_classification_codes_all_levels[
            "household_composition_code"
        ].unique()
    )
]

In [8]:
unique_areas = df["area_code"].drop_duplicates()


unique_areas = unique_areas[
    ~unique_areas.isin(
        areas_all_levels["area_code"].unique()
    )
]

In [9]:
unique_num_beds_comps = df["num_bedrooms"].drop_duplicates()


# unique_hhld_comps = unique_hhld_comps[
#     ~unique_hhld_comps.astype(int).isin(
#         household_composition_classification_codes_all_levels["value"].unique()
#     )
# ]

In [10]:
# # Define the path to your CSV file
# file_path = "data/raw/STATSNZ,CEN23_FHH_038,1.0+all.csv"

# # Read the CSV file into a pandas DataFrame
# df = pd.read_csv(file_path, dtype='str')

# # --- Step 1: Automatically detect and split columns with 'code:name' format ---
# columns_to_drop = []
# original_columns = df.columns.copy()
# for col_name in original_columns:
#     if df[col_name].dtype == "object":
#         non_empty_values = df[col_name].dropna()
#         if not non_empty_values.empty:
#             first_value = non_empty_values.iloc[0]
#             if ":" in str(first_value):
#                 new_cols = df[col_name].str.split(":", n=1, expand=True)
#                 df[col_name + "_code"] = new_cols[0].str.strip()
#                 df[col_name + "_name"] = new_cols[1].str.strip()
#                 columns_to_drop.append(col_name)
# if columns_to_drop:
#     df = df.drop(columns=columns_to_drop)

# # --- Step 2: Analyze the lengths of the area codes ---
# area_code_col = None
# for col in df.columns:
#     if "area" in col.lower() and col.endswith("_code"):
#         area_code_col = col
#         break
# if area_code_col:
#     df["code_length"] = df[area_code_col].str.len()

In [11]:
# keep_df = df[[
#     'CEN23_YEAR_001: Census year_name',
#     'CEN23_GEO_013: Area_code',
#     'CEN23_HUH_001: Household composition by number of usual residents_code',
#     'CEN23_NRD_002: Number of bedrooms',
#     'OBS_VALUE'
#              ]]

# rename_map = {
#     'CEN23_YEAR_001: Census year_name':"census_year",
#     "CEN23_GEO_013: Area_code": "area_code",
#     "CEN23_HUH_001: Household composition by number of usual residents_code": "household_composition_code",
#     "CEN23_NRD_002: Number of bedrooms": "num_bedrooms",
# }

# keep_df.rename(columns=rename_map, inplace=True)

# keep_df.to_csv('data/processed/processed_data.csv')

In [12]:
df = df[
    df["area_code"].isin(areas_all_levels["area_code"].unique())
    & df["household_composition_code"].isin(
        household_composition_classification_codes_all_levels[
            "household_composition_code"
        ].unique()
    )
    & df["num_bedrooms"].isin(number_bedrooms_classification["num_bedrooms_code"])
]

df = df.merge(areas_all_levels).merge(household_composition_classification_codes_all_levels).merge(number_bedrooms_all_levels)

In [13]:
filtered_df = df[
    (df['num_bedrooms_level'] == 'num_bedrooms_l1_code')
    & (df['area_level'] == 'Total')
    & (df['hhld_comp_level'] == 'Total')
    ]

filtered_df = filtered_df[['census_year','num_bedrooms','OBS_VALUE']]

print(filtered_df)

        census_year num_bedrooms OBS_VALUE
2480992        2013           01     81576
2480993        2013           02    280254
2480994        2013           03    656304
2480995        2013           04    345603
2480996        2013           05    108042
2480998        2013           99     78111
4961992        2018           01    101334
4961993        2018           02    316014
4961994        2018           03    719424
4961995        2018           04    395157
4961996        2018           05    121077
4961998        2018           99       777
7442992        2023           01    120585
7442993        2023           02    346641
7442994        2023           03    748236
7442995        2023           04    427845
7442996        2023           05    136542
7442998        2023           99       678


In [14]:
lowest_level_data = df[
    (df["num_bedrooms_level"] == "num_bedrooms_l1_code")
    & (df["area_level"] == "SA22023_code")
    & (df["hhld_comp_level"] == "hhld_comp_l3_code")
    & (df['census_year'] == '2023')
]

lowest_level_data = lowest_level_data[
    ["area_code", "household_composition_code", "num_bedrooms", "OBS_VALUE"]
]

lowest_level_data.to_csv('data/processed/sa2_level_data.csv')

In [15]:
ta_level_data = df[
    (df["num_bedrooms_level"] == "num_bedrooms_l1_code")
    & (df["area_level"] == "TA2023_code")
    & (df["hhld_comp_level"] == "hhld_comp_l3_code")
    & (df["census_year"] == "2023")
]

ta_level_data = ta_level_data[
    ["area_code", "household_composition_code", "num_bedrooms", "OBS_VALUE"]
]

ta_level_data.to_csv('data/processed/ta_level_data.csv')

# ta_level_data["OBS_VALUE"] = pd.to_numeric(ta_level_data["OBS_VALUE"])

In [16]:
summarised_sa2_at_ta = lowest_level_data.merge(
    sa2_ta_concordance, left_on="area_code", right_on="SA22023_code"
)

summarised_sa2_at_ta["OBS_VALUE"] = pd.to_numeric(summarised_sa2_at_ta["OBS_VALUE"])

summarised_sa2_at_ta = (
    summarised_sa2_at_ta
    .groupby(
        ["household_composition_code", "num_bedrooms", "TA2023_code"], as_index=False
    )
    .sum()
    .loc[:, ["household_composition_code", "num_bedrooms", "TA2023_code", "OBS_VALUE"]]
    .rename(columns = {'TA2023_code':'area_code','OBS_VALUE':'SA2_SUM'})
)

In [17]:
combined = ta_level_data.merge(summarised_sa2_at_ta)

In [19]:
national_level_data = df[
    (df["num_bedrooms_level"] == "num_bedrooms_l1_code")
    & (df["area_level"] == "Total")
    & (df["hhld_comp_level"] == "hhld_comp_l3_code")
    & (df["census_year"] == "2023")
]

national_level_data = national_level_data[
    ["area_code", "household_composition_code", "num_bedrooms", "OBS_VALUE"]
]

national_level_data.to_csv("data/processed/national_level_data.csv")